In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import cmaps

2021.07.06

水汽通量垂直积分的绘图部分

从 shumflux_vi_count.ipynb 的预存数据中读取，进行绘图

2021.07.22

修改单位为kg * m-1 * s-1

2021.08.15

修改到更大的区域上来进行绘制，并且在前一步积分过程中将NAN设置为0，消去此前的很多NAN区域

2022.02.08

将分析时间段剪短到MJJA

In [ ]:
dir_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/dyn/shum_flux/"
mod_list    = ['obs', 'vr', 'rcm']
season_list = ['mjja']
# 读取水汽通量的垂直积分
uflux_vi = {}
vflux_vi = {}

for imod in mod_list:
    uflux_vi[imod]  = {}
    vflux_vi[imod]  = {}

    for iseason in season_list:
        uflux_vi[imod][iseason]  = xr.open_dataset(dir_in + "shum_uflux_"+iseason+".nc")[imod]
        vflux_vi[imod][iseason]  = xr.open_dataset(dir_in + "shum_vflux_"+iseason+".nc")[imod]

In [ ]:
# 计算绘图的平均值
uflux_vi_mean = {} # 平均u方向水汽通量
vflux_vi_mean = {} # 平均v方向水汽通量
absflux_vi_mean = {} # 水汽通量大小，平均图
for imod in mod_list:
    uflux_vi_mean[imod] = {}
    vflux_vi_mean[imod] = {}
    absflux_vi_mean[imod] = {}
    for iseason in season_list:
        uflux_vi_mean[imod][iseason] = uflux_vi[imod][iseason].mean(axis=0) * 1e2 / 9.8 # 转换单位
        vflux_vi_mean[imod][iseason] = vflux_vi[imod][iseason].mean(axis=0) * 1e2 / 9.8
        absflux_vi_mean[imod][iseason] = np.sqrt(uflux_vi_mean[imod][iseason] ** 2 + vflux_vi_mean[imod][iseason] **2) 
# flux 等值线绘制大小

#### 计算水汽输送的相关系数 均方根误差

In [ ]:
import skill_metrics as sm
flux_1d = {}
two_fluxs_dict = {'uflux_vi_mean':uflux_vi_mean, 'vflux_vi_mean':vflux_vi_mean}
for mod_name in ['obs', 'vr', 'rcm']:
    flux_1d[mod_name] = {}
    for vartype in two_fluxs_dict:
        temp_obs = two_fluxs_dict[vartype]['obs']['mjja'].values
        temp_mod = two_fluxs_dict[vartype][mod_name]['mjja'].values

            # 获取1D的平均态 并以RCM为基础进行NAN的MASK
        index_not_nan = ( ~np.isnan(two_fluxs_dict[vartype]['obs']['mjja'].values)) &\
            (~np.isnan(two_fluxs_dict[vartype]['vr']['mjja'].values)) &\
            (~np.isnan(two_fluxs_dict[vartype]['rcm']['mjja'].values) )
            
        var_1d_temp = xr.where(index_not_nan, two_fluxs_dict[vartype][mod_name]['mjja'], np.nan).values.ravel()
        flux_1d[mod_name][vartype] = var_1d_temp[~np.isnan(var_1d_temp)]


metrics = {}
for mod_name in ['vr','rcm']:
    metrics[mod_name] = {}
    metrics[mod_name] = pd.DataFrame(index = two_fluxs_dict.keys(), columns=['Bias','RMSE','Corr.']) 
    for vartype in two_fluxs_dict:
        temp_obs = flux_1d['obs'][vartype]
        temp_mod = flux_1d[mod_name][vartype]
        metrics[mod_name].loc[vartype]['Bias'] = sm.bias(temp_mod, temp_obs)
        metrics[mod_name].loc[vartype]['RMSE'] = sm.rmsd(temp_mod, temp_obs)
        metrics[mod_name].loc[vartype]['Corr.'] = np.corrcoef(temp_mod, temp_obs)[0,1]

        # # 对style进行调整，用于后续展示表格
        # # https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
    # metrics_std[vartype] = metrics_std[vartype].style.format("{:,.3f}")

In [ ]:
metrics['rcm']

# 绘图部分

In [ ]:
# 整理绘图所使用的坐标变量
lat  = uflux_vi['vr']['mjja'].latitude
lon  = uflux_vi['vr']['mjja'].longitude
# time = uflux_vi['vr']['jja'].Time
# time_am = uflux_vi['vr']['am'].Time
time_mjja = uflux_vi['vr']['mjja'].Time
# vr_uflux_sum_addcoords = xr.DataArray(vr_uflux_sum, coords=[time,lat,lon], name = "qv uflux").mean('Time')
# vr_vflux_sum_addcoords = xr.DataArray(vr_vflux_sum, coords=[time,lat,lon], name = "qv vflux").mean('Time')
# vr_flux_all = np.sqrt(vr_uflux_sum_addcoords ** 2 + vr_vflux_sum_addcoords ** 2 )

In [ ]:
# 调整cmap，去掉gist_ncar 中深蓝色的部分
# https://stackoverflow.com/questions/18926031/how-to-extract-a-subset-of-a-colormap-as-a-new-colormap-in-matplotlib

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import cmaps
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

arr = np.linspace(0, 50, 100).reshape((10, 10))
fig, ax = plt.subplots(ncols=2)

cmap = plt.get_cmap('gist_ncar')
# cmap = cmaps.GMT_wysiwygcont
# cmap = cmaps.WhiteBlueGreenYellowRed

new_cmap = truncate_colormap(cmap, 0.1, 1.)
# new_cmap = truncate_colormap(cmap, 0., 1.)
ax[0].imshow(arr, interpolation='nearest', cmap=cmap)
ax[1].imshow(arr, interpolation='nearest', cmap=new_cmap)
plt.show()

In [ ]:
# 国内政区图的绘制
# Load the border data, CN-border-La.dat is download from
# https://gmt-china.org/data/CN-border-La.dat
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
from matplotlib.font_manager import FontProperties

from matplotlib import pyplot as plt
import proplot as plot
import cmaps

In [ ]:
# cmap = "coolwarm"
# cmap = "gist_ncar"
cmap = new_cmap

# ----- get filter vars coords-----

# cticks = np.arange(0,700,20)
cticks = np.concatenate((plot.arange(20,200,20),plot.arange(240,600,40)))

#----- create plot -----
# fig, axs = plot.subplots(ncols=3,nrows=1, proj=('cyl'), width = '190mm',height = '115mm', share = True)
shape_array = [[0,1,1,0],[2,2,3,3]]
fig, axs = plot.subplots(shape_array, proj=('cyl'), width = '130mm',height = '115mm', share = True)

##---- 直接绘图，从边界文件添加
turn_on_border = True
if(turn_on_border):
    # for ax_ind in axs:
    # for line in borders:
        # axs.plot(line[0::100], line[1::100], lw = 0.5, color='gray',transform=ccrs.Geodetic())
        # axs.plot(line[0::10], line[1::10], lw = 0.4, color='black',transform=ccrs.Geodetic())
    ##---- 使用shp文件添加
        ## shapefile数据下载的位置：
    ## http://gaohr.win/site/blogs/2017/2017-04-18-GIS-basic-data-of-China.html
    # world_border_shapefile = "/m2data2/yycheng/data_stage/CN-border/World/country.shp"
    river_border_shapefile =  "/raid52/yycheng/MPAS/REFERENCE/MODEL_CONSTANT/R1/" + "hyd1_4l.shp"
    southsea_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "southsea_island.shp"
    ninelines_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "nine_lines.shp"
    ## 来源： 沛沛的诸省 + 诸岛
    bou24p_shapefile     = "/m2data2/yycheng/data_stage/CN-border/peipeihelp/" + "bou2_4p.shp"
    ## 来源： https://www.resdc.cn/data.aspx?DATAID=200
    province_shapefile     = "/m2data2/yycheng/data_stage/CN-border/CN-sheng/" + "change_proj_CN-sheng-A.shp"

    for ax in axs:
        # world     = shpreader.Reader(world_border_shapefile).geometries()
        # river     = shpreader.Reader(river_border_shapefile).geometries()
        river     = shpreader.Reader(river_border_shapefile, encoding = 'gbk')
        # bou24p    = shpreader.Reader(bou24p_shapefile).geometries()
        ninelines = shpreader.Reader(ninelines_shapefile).geometries()
        province  = shpreader.Reader(province_shapefile).geometries()
        # ax.add_geometries(world, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.4, zorder=1)
        # ax.add_geometries(bou24p, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 沛沛map
        ax.add_geometries(province, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 地资所
        ax.add_geometries(ninelines, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1)
        # ax.add_geometries(river, ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)
        # 绘制部分的shapefile
        for region in river.records():
            if (region.attributes['NAME'] in ['黄河','长江']):
                # print("----- draw river! -----")
                # 此处需要使用 [] 让region.geometry可以迭代
                ax.add_geometries([region.geometry], ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.6, zorder=1)


scale_num = 3600
#-----vr-----
axs[0].format(title='ERA-Interim')
# axs[0].format(ltitle='MJJA')
m_contour= axs[0].contourf(lon, lat, absflux_vi_mean['obs']['mjja'], levels = cticks, cmap=cmap, norm = 'segmented', extend = 'both')
m_quiver = axs[0].quiver(lon[::10], lat[::10], uflux_vi_mean['obs']['mjja'][::10,::10], vflux_vi_mean['obs']['mjja'][::10,::10],scale = scale_num)

axs[1].format(title='VR')
# axs[1].format(ltitle='MJJA')
m_contour= axs[1].contourf(lon, lat, absflux_vi_mean['vr']['mjja'], levels = cticks, cmap=cmap, norm = 'segmented', extend = 'both')
m_quiver = axs[1].quiver(lon[::10], lat[::10], uflux_vi_mean['vr']['mjja'][::10,::10], vflux_vi_mean['vr']['mjja'][::10,::10],scale = scale_num)
#-----rcm -----
axs[2].format(title='RCM')
# axs[2].format(ltitle='MJJA')
m_contour= axs[2].contourf(lon, lat, absflux_vi_mean['rcm']['mjja'], levels = cticks, cmap=cmap, norm = 'segmented', extend = 'both')
m_quiver = axs[2].quiver(lon[::10], lat[::10], uflux_vi_mean['rcm']['mjja'][::10,::10], vflux_vi_mean['rcm']['mjja'][::10,::10],scale = scale_num)
# 矢量图 quiverkey设置
axs[2].quiverkey(m_quiver, 0.5, 1.15, label=r"$400 kg \cdot m^{-1} \cdot s^{-1}$", U=400, labelpos="E")

fig.colorbar(m_contour, cmap=cmap,loc='b', width=0.1,ticklabelsize=5,ticks=cticks[::],label="[" + r"$kg \cdot m^{-1} \cdot s^{-1}$" + "]", extend = 'both')
# format作为整个proplot集中对属性进行修改的method
axs.format(
abc=True,
abcloc = "ul",
lonlim=(70, 140), latlim=(5, 60), 
#----- 地图底图设置 -----
# reso = 'x-hi',
reso = 'med',
# coast = False,
coast = True,
coastlinewidth = 0.4,
borders = False,
lakes = False,
land  = False,
ocean = False,
# cartopyautoextent = True, 
# borderslinewidth=.5,
# borders = True,

#-----GEO axis-----
labels = True,
gridlabelsize = 5,
gridminor = False,

longrid  = True,
latgrid  = True,
# latlines = 10,
# lonlines = 10,
lonlocator = np.arange(70,150,10),
latlocator = np.arange(15,65,10),
lonminorlocator = np.arange(70,140+2,2),
latminorlocator = np.arange(15,55+2,2),
#-----line label-----
# linewidth = 0.5,
# suptitle="925-200hPa vertical specific humidity flux integration",
 titleweight='bold',
)
# #----- add contour tick labels -----
# # axs.clabel(m, inline=True, fontsize=5, colors = 'k', fmt='%4.0f',use_clabeltext=True, inline_spacing=0)
# fig.colorbar(m_contour, loc='b', cmap=cmap, width=0.1)

# ----- add table on axis -----
# 参考回答：https://stackoverflow.com/questions/54150557/how-to-show-dataframe-index-name-on-a-matplotlib-table
# 使用bbox调整表格的大小
plot_ind = 0
for mod_name in ['obs','vr', 'rcm']:
    if (mod_name == 'obs'):
        plot_ind +=1
        continue

    plot_table = metrics[mod_name].applymap("{:,.2f}".format)
    table = axs[plot_ind].table(cellText=np.matrix.round(plot_table.values.astype('float'),3),rowLabels=["$\\bf{Uflux}$","$\\bf{Vflux}$"], \
                colLabels=plot_table.columns, loc = 'center',colColours=['gainsboro'] * 3,\
                colWidths = [0.13]*3, bbox = [0.13, .78, 0.35, 0.15], zorder = 10)
    table.auto_set_font_size(False) # 关闭自动网格字体大小设置
    table.set_fontsize(5.) # 设置数字字体大小
    # table.set_fontweight('roman')
                #  colColours=['gainsboro'] * len(plot_table), colLabels=plot_table.columns, loc='center',
                #  colWidths= [0.12]*(len(plot_table.columns)))
    for (row, col), cell in table.get_celld().items():
        if ((row == 0)):
            cell.set_text_props(fontproperties=FontProperties(weight='bold', size = 5.))
    plot_ind +=1

fig.patch.set_facecolor('white')
fig.savefig('./output_pic/waterfluxvi.2022.05.04.jpg', dpi=600, facecolor= "white", bbox_inches='tight')